In [ ]:
import sys
import os

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from PyQt5.QtWidgets import QMainWindow, QWidget, QGridLayout, QVBoxLayout, QHBoxLayout, QApplication, QMessageBox
from PyQt5.QtCore import Qt
from optic.gui.app_setup import setupMainWindow
from optic.gui.app_style import applyAppStyle
from optic.manager import WidgetManager, ConfigManager, DataManager, ControlManager, LayoutManager, initManagers


class BehaviorCameraGUI(QMainWindow):
    """行動実験用カメラ制御GUI"""
    
    def __init__(self):
        APP_NAME = "BEHAVIOR_CAMERA"
        QMainWindow.__init__(self)
        self.widget_manager, self.config_manager, self.data_manager, self.control_manager, self.layout_manager = initManagers(
            WidgetManager(), ConfigManager(), DataManager(), ControlManager(), LayoutManager()
        )
        self.config_manager.setCurrentApp(APP_NAME)
        self.app_keys = self.config_manager.gui_defaults["APP_KEYS"]
        self.app_key_pri = self.app_keys[0]

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)

        # 変数の初期化
        self.camera = None
        self.is_capturing = False

        # UI初期化
        self.initUI()
    
    def initUI(self):
        """UIの初期化"""
        # 中央ウィジェット
        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)
        
        # メインレイアウト
        self.layout_main = QGridLayout(self.central_widget)
        
        # セクションレイアウトの配置
        # 左側上部：Config
        self.layout_main.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
        # 左側下部：Capture
        self.layout_main.addLayout(self.makeLayoutSectionLeftLower(), 1, 0, 1, 1)
        # 右側全体：Image
        self.layout_main.addLayout(self.makeLayoutSectionRight(), 0, 1, 2, 1)
        
        # レイアウトの列幅比率を設定
        self.layout_main.setColumnStretch(0, 1)
        self.layout_main.setColumnStretch(1, 3)
    

    
    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    
    def makeLayoutComponentSaveDirectory(self):
        """Save Directory, Prefix, Move Destination の設定"""
        layout = QVBoxLayout()
        
        # Save Directory
        layout_savedir = QHBoxLayout()
        layout_savedir.addWidget(self.widget_manager.makeWidgetLabel(
            key="savedir_label",
            label="Save Directory:",
            use_global_style=True
        ))
        layout_savedir.addWidget(self.widget_manager.makeWidgetLineEdit(
            key="savedir",
            text_set="/path/to/save",
            width_fix=400,
            use_global_style=True
        ))
        layout_savedir.addWidget(self.widget_manager.makeWidgetButton(
            key="savedir_browse",
            label="Browse",
            func_=None,
            use_global_style=True
        ))
        layout.addLayout(layout_savedir)
        
        # Save Directory Prefix
        layout_prefix = QHBoxLayout()
        layout_prefix.addWidget(self.widget_manager.makeWidgetLabel(
            key="savedir_prefix_label",
            label="Save Directory Prefix:",
            use_global_style=True
        ))
        layout_prefix.addWidget(self.widget_manager.makeWidgetComboBox(
            key="savedir_prefix",
            items=["dlc-pupil", "HoG-ActiveWhisking", "HoG-Sniffing"],
            idx_default=0,
            use_global_style=True
        ))
        layout_prefix.addStretch()
        layout.addLayout(layout_prefix)
        
        # Move Destination
        layout_movedst = QHBoxLayout()
        layout_movedst.addWidget(self.widget_manager.makeWidgetLabel(
            key="movedst_label",
            label="Move Destination:",
            use_global_style=True
        ))
        layout_movedst.addWidget(self.widget_manager.makeWidgetLineEdit(
            key="movedst",
            text_set="/path/to/move",
            width_fix=400,
            use_global_style=True
        ))
        layout_movedst.addWidget(self.widget_manager.makeWidgetButton(
            key="movedst_browse",
            label="Browse",
            func_=None,
            use_global_style=True
        ))
        layout.addLayout(layout_movedst)
        
        return layout
    
    def makeLayoutComponentCameraConfig(self):
        """Camera Config (fps, width, height, offset, gain, exposure)"""
        layout = QVBoxLayout()
        
        # セクションタイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="camera_config_title",
            label="Camera Settings",
            bold=True,
            use_global_style=False
        ))
        
        # カメラパラメータ
        params = [
            ("fps", "FPS:", "60.0"),
            ("width", "Width:", "1280"),
            ("height", "Height:", "1024"),
            ("offsetx", "Offset X:", "0"),
            ("offsety", "Offset Y:", "0"),
            ("gain", "Gain:", "12.0"),
            ("exposure_time", "Exposure Time:", "2000"),
        ]
        
        for key, label_text, default_value in params:
            layout_param = QHBoxLayout()
            layout_param.addWidget(self.widget_manager.makeWidgetLabel(
                key=f"camera_{key}_label",
                label=label_text,
                use_global_style=True
            ))
            layout_param.addWidget(self.widget_manager.makeWidgetLineEdit(
                key=f"camera_{key}",
                text_set=default_value,
                width_fix=100,
                use_global_style=True
            ))
            layout_param.addStretch()
            layout.addLayout(layout_param)
        
        return layout
    
    def makeLayoutComponentCapturePreview(self):
        """Capture Single shot, Play, FPS表示"""
        layout = QVBoxLayout()
        
        # ボタン行
        layout_buttons = QHBoxLayout()
        layout_buttons.addWidget(self.widget_manager.makeWidgetButton(
            key="capture_single",
            label="Capture Single",
            func_=None,
            use_global_style=True
        ))
        layout_buttons.addWidget(self.widget_manager.makeWidgetButton(
            key="play",
            label="Play",
            func_=None,
            use_global_style=True
        ))
        layout_buttons.addStretch()
        layout.addLayout(layout_buttons)
        
        # FPS表示
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="fps_display",
            label="FPS: 0",
            use_global_style=True
        ))
        
        return layout
    
    def makeLayoutComponentCaptureControl(self):
        """Start Capture with Bpod, Move Video Files, Exit"""
        layout = QVBoxLayout()
        
        # 撮影開始ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="start_capture_bpod",
            label="Start Capture with Bpod",
            func_=None,
            use_global_style=True
        ))
        
        # ステータス表示
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="status_display",
            label="Status: Standby",
            use_global_style=True
        ))
        
        # ファイル移動ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="move_video",
            label="Move Video Files",
            func_=None,
            use_global_style=True
        ))
        
        # 終了ボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(
            key="exit",
            label="Exit",
            func_=self.close,
            use_global_style=True
        ))
        
        return layout
    
    def makeLayoutComponentImageDisplay(self):
        """カメラ画像表示用キャンバス"""
        layout = QVBoxLayout()
        
        # 画像表示エリア
        label_image = self.widget_manager.makeWidgetLabel(
            key="camera_preview",
            label="Camera Preview Area",
            align=Qt.AlignCenter,
            use_global_style=True
        )
        label_image.setMinimumSize(1280, 1024)
        label_image.setStyleSheet("border: 1px solid black; background-color: #f0f0f0;")
        layout.addWidget(label_image)
        
        return layout
    
    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # Left Upper Section
    def makeLayoutSectionLeftUpper(self):
        """Config全体（Save Directory + Camera Config）"""
        layout = QVBoxLayout()
        
        # title
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="config_title",
            label="Configuration",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentSaveDirectory())
        layout.addLayout(self.makeLayoutComponentCameraConfig())
        layout.addStretch()
        
        return layout
    
    # Left Lower Section
    def makeLayoutSectionLeftLower(self):
        """Capture全体（Preview + Control）"""
        layout = QVBoxLayout()
        
        # タイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="capture_title",
            label="Capture Control",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentCapturePreview())
        layout.addLayout(self.makeLayoutComponentCaptureControl())
        layout.addStretch()
        
        return layout
    
    # Right Section
    def makeLayoutSectionRight(self):
        """Image表示"""
        layout = QVBoxLayout()
        
        # タイトル
        layout.addWidget(self.widget_manager.makeWidgetLabel(
            key="image_title",
            label="Camera View",
            font_size=14,
            bold=True,
            use_global_style=False
        ))
        
        # コンポーネント
        layout.addLayout(self.makeLayoutComponentImageDisplay())
        
        return layout


# 実行用
if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
    applyAppStyle(app)
    gui = BehaviorCameraGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0